In [123]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sentence_transformers import SentenceTransformer
import umap
from sklearn.cluster import KMeans
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
import re
from sklearn.metrics.pairwise import cosine_similarity
from utils import jaccard_similarity_score
import hdbscan
import networkx as nx

In [124]:
df = pd.read_csv('wiki_movie_plots_deduped.csv')
df = df[df['Origin/Ethnicity'].isin(['Bollywood','Malayalam','Tamil','American'])]
df = df[df['Release Year'] >= 2010].sample(500)
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
27085,2017,Mom,Bollywood,Ravi Udyawar,"Sridevi, Akshaye Khanna & Nawazuddin Siddiqui",thriller/drama,https://en.wikipedia.org/wiki/Mom_(film),Biology teacher Devki is vivacious and popular...
28239,2011,Priyappetta Nattukare (പ്രിയപ്പെട്ട നാട്ടുകാരേ),Malayalam,Sreejith Paleri,"Kalabhavan Mani, Bala, Lakshmi Sharma",political,https://en.wikipedia.org/wiki/Priyappetta_Natt...,Priyappetta Nattukare is a political thriller ...
16068,2011,The Lincoln Lawyer,American,Brad Furman,"Matthew McConaughey, Marisa Tomei, Ryan Philli...","mystery, suspense thriller",https://en.wikipedia.org/wiki/The_Lincoln_Lawy...,Criminal defense attorney Mickey Haller (Matth...
31309,2015,Nannbenda,Tamil,Jagdish,"Udhayanidhi Stalin, Nayantara, Santhanam",romantic comedy,https://en.wikipedia.org/wiki/Nannbenda,"Sathya is an unemployed youth from Thanjavur, ..."
28280,2012,Dhanyam (ധന്യം),Malayalam,Jayalal,"Pratheesh Nandan, Pooja",romance,https://en.wikipedia.org/wiki/Dhanyam,Dhanyam tells about the virtues of Kerala vill...


In [ ]:
import spacy
import pandas as pd
from tqdm import tqdm

# Load the spaCy model once
nlp = spacy.load("en_core_web_md")

def mask_named_entities(text):
    """
    Uses spaCy to identify PERSON entities and replaces them with a placeholder.
    """
    if not isinstance(text, str):
        return text  # Handles NaN or non-string inputs

    doc = nlp(text)
    
    # Create a list of tuples: (start_char, end_char, replacement_string)
    # The replacement string is our placeholder
    replacements = []
    
    for ent in doc.ents:
        # We only care about names (PERSON entities)
        if ent.label_ == "PERSON":
            replacements.append((ent.start_char, ent.end_char, "[PERSON_NAME]"))

    # Apply replacements from end to start to avoid shifting indices
    replacements.sort(key=lambda x: x[0], reverse=True)
    
    masked_text = list(text)
    for start, end, repl in replacements:
        # Replace the characters in the range with the placeholder
        masked_text[start:end] = repl
    
    return "".join(masked_text)

print("Starting Named Entity Masking...")

# Use tqdm for a progress bar since NER can take a while on a large dataset
tqdm.pandas()
df['Plot_Masked'] = df['Plot'].str.replace(r'\([a-zA-Z\s]*\)', '', regex=True).progress_apply(mask_named_entities)

print("Masking complete.")

# 3. Create your new 'combined' column using the masked plot
df['combined_masked'] = (
    df['Origin/Ethnicity'] + ' ' + 
    df['Title'] + ' ' + 
    df['Genre'] + ' ' + 
    df['Plot_Masked']
)

Starting Named Entity Masking...


100%|██████████| 500/500 [02:37<00:00,  3.18it/s]

Masking complete.


In [114]:
df[['Plot', 'Plot_Masked']].head()

,Plot,Plot_Masked
27105,Kishen Mohan Girhotra (Farhan Akhtar) is a you...,[PERSON_NAME] is a young man who belongs to M...
16743,Laurel and Audrey (Zoe Kazan) are identical tw...,Laurel and [PERSON_NAME] are identical twins....
27081,"In the city of Lucknow, Gattu (Rajkummar Rao),...","In the city of Lucknow, Gattu , a jobless UPSC..."
15821,"Olive Penderghast, a 17-year-old girl living i...","[PERSON_NAME], a 17-year-old girl living in Oj..."
26785,The film is about a girl named Tanya (Esha Deo...,The film is about a girl named [PERSON_NAME] ...


In [115]:
df['combined'] = df['Origin/Ethnicity'] + ' ' + df['Title'] + ' ' + df['Genre'] + ' ' + df['Plot']

input = df['Plot_Masked'].tolist()

vectorizer_model = CountVectorizer(stop_words="english", max_df = 0.5, min_df = 0.2)
emb_minilm = SentenceTransformer("all-MiniLM-L6-v2")
umap_model = umap.UMAP(
    n_neighbors=5,
    min_dist=0.01, 
    random_state=0
)

# clustering_model = KMeans(
#     n_clusters=12
# )

MIN_CLUSTER_SIZE= 15
MIN_SAMPLES = 1
clustering_model = hdbscan.HDBSCAN(
    min_cluster_size=MIN_CLUSTER_SIZE,
    min_samples=MIN_SAMPLES,
    prediction_data=True
)

ctfidf_model = ClassTfidfTransformer(bm25_weighting=True, reduce_frequent_words=True)

representation_model = KeyBERTInspired()

topic_model = BERTopic(language="english", 
                       embedding_model=emb_minilm,
                       vectorizer_model=vectorizer_model, 
                       umap_model=umap_model,
                       hdbscan_model=clustering_model,
                       ctfidf_model=ctfidf_model,
                       representation_model=representation_model,
                       calculate_probabilities=True, 
                       verbose=True)
topics, probs = topic_model.fit_transform(input)

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: b0229608-5a01-4033-bbd5-964e639c29cc)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


2025-11-30 15:39:46,111 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 16/16 [00:15<00:00,  1.01it/s]
2025-11-30 15:40:02,016 - BERTopic - Embedding - Completed ✓
2025-11-30 15:40:02,021 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-30 15:40:03,980 - BERTopic - Dimensionality - Completed ✓
2025-11-30 15:40:04,002 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-30 15:40:04,267 - BERTopic - Cluster - Completed ✓
2025-11-30 15:40:04,307 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-30 15:40:07,739 - BERTopic - Representation - Completed ✓


In [116]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,119,-1_murderer_samantha_betrayed_criminals,"[murderer, samantha, betrayed, criminals, betr...",[Royce awakens to find himself parachuting int...
1,0,50,0_suspecting_murdering_stealing_related,"[suspecting, murdering, stealing, related, rob...","[In 2002, former Pittsburgh Steelers center [P..."
2,1,50,1_detective_lawyer_fiancé_julia,"[detective, lawyer, fiancé, julia, coffin, nei...",[Julia Banks is being questioned by a detectiv...
3,2,41,2_assassinate_kidnapping_criminals_colleague,"[assassinate, kidnapping, criminals, colleague...",[The story begins with an animated interpretat...
4,3,36,3_cinema_exorcism_passenger_drama,"[cinema, exorcism, passenger, drama, maid, bet...",[The movie begins with a guy called [PERSON_NA...
5,4,36,4_hostage_victims_passenger_distraught,"[hostage, victims, passenger, distraught, inca...",[[PERSON_NAME] is a Los Angeles Fire Departmen...
6,5,35,5_orphan_prostitute_sanjay_dumped,"[orphan, prostitute, sanjay, dumped, character...",[Story 1: Delhi\r\nDimple Chaddha is a spoilt...
7,6,32,6_heroine_drama_actor_prostitute,"[heroine, drama, actor, prostitute, bollywood,...",[A young man [PERSON_NAME] returns to India f...
8,7,26,7_ambushed_robbers_imprisoned_sheriff,"[ambushed, robbers, imprisoned, sheriff, aband...","[Draenor, the homeworld of the orcs, is being ..."
9,8,25,8_kidnapping_thief_identifies_kidnappers,"[kidnapping, thief, identifies, kidnappers, cl...",[[PERSON_NAME] is the honest police sub-inspe...


In [117]:
topic_info = topic_model.get_topic_info()
topic_rep_map = topic_info.set_index("Topic")["Representation"].to_dict()
topic_info[topic_info['Topic'] == 0]

,Topic,Count,Name,Representation,Representative_Docs
1,0,50,0_suspecting_murdering_stealing_related,"[suspecting, murdering, stealing, related, rob...","[In 2002, former Pittsburgh Steelers center [P..."


In [118]:
emb_model = SentenceTransformer("all-MiniLM-L6-v2") 
embeddings = emb_model.encode(input)
df['Embedding'] = list(embeddings)

df['Topic'] = topics
# df['Topic_Probs'] = probs
df['Topic_Rep'] = df['Topic'].map(topic_rep_map)
# df[~df['Topic_Probs'].isna()].head()
# df[df['Title'].isin(['Spy Kids: All the Time in the World', 'Cars 2'])]
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Plot_Masked,combined_masked,combined,Embedding,Topic,Topic_Rep
27105,2017,Ranjit Tiwari,Bollywood,"Farhan Akhtar, Diana Penty, Deepak Dobriyal, G...",Drama,viacom 18 motion pictures,https://en.wikipedia.org/wiki/Lucknow_Central,Kishen Mohan Girhotra (Farhan Akhtar) is a you...,[PERSON_NAME] is a young man who belongs to M...,Bollywood Ranjit Tiwari viacom 18 motion pictu...,Bollywood Ranjit Tiwari viacom 18 motion pictu...,"[-0.033787288, 0.017728621, -0.06242119, -0.08...",-1,"[murderer, samantha, betrayed, criminals, betr..."
16743,2014,The Pretty One,American,Jenée LaMarque,Zoe Kazan\r\nJake Johnson\r\nRon Livingston\r\...,comedy,https://en.wikipedia.org/wiki/The_Pretty_One,Laurel and Audrey (Zoe Kazan) are identical tw...,Laurel and [PERSON_NAME] are identical twins....,American The Pretty One comedy Laurel and [PER...,American The Pretty One comedy Laurel and Audr...,"[-0.095008805, 0.0151873985, 0.022199338, -0.0...",0,"[suspecting, murdering, stealing, related, rob..."
27081,2017,Behen Hogi Teri,Bollywood,Ajay K Pannalal,Rajkummar Rao & Shruti Haasan,romance/comedy,https://en.wikipedia.org/wiki/Behen_Hogi_Teri,"In the city of Lucknow, Gattu (Rajkummar Rao),...","In the city of Lucknow, Gattu , a jobless UPSC...",Bollywood Behen Hogi Teri romance/comedy In th...,Bollywood Behen Hogi Teri romance/comedy In th...,"[-0.09154939, 0.090776615, -0.08054858, -0.004...",-1,"[murderer, samantha, betrayed, criminals, betr..."
15821,2010,Easy A,American,Will Gluck,"Emma Stone, Penn Badgley, Amanda Bynes, Thomas...","comedy, teen",https://en.wikipedia.org/wiki/Easy_A,"Olive Penderghast, a 17-year-old girl living i...","[PERSON_NAME], a 17-year-old girl living in Oj...","American Easy A comedy, teen [PERSON_NAME], a ...","American Easy A comedy, teen Olive Penderghast...","[-0.10341715, 0.09304789, -0.00087627413, -0.0...",-1,"[murderer, samantha, betrayed, criminals, betr..."
26785,2011,Tell Me O Kkhuda,Bollywood,"Mayur Puri, Hema Malini","Esha Deol, Vinod Khanna, Rishi Kapoor, Arjan B...",drama,https://en.wikipedia.org/wiki/Tell_Me_O_Kkhuda,The film is about a girl named Tanya (Esha Deo...,The film is about a girl named [PERSON_NAME] ...,Bollywood Tell Me O Kkhuda drama The film is a...,Bollywood Tell Me O Kkhuda drama The film is a...,"[-0.06768297, 0.04301227, -0.07049474, 0.02944...",3,"[cinema, exorcism, passenger, drama, maid, bet..."


In [119]:
import numpy as np

def calculate_similarity(query_movie, candidate_movie):

    # Define the weights to enforce the strict hierarchy
    WEIGHT_ORIGIN = 1
    WEIGHT_GENRE = 1
    WEIGHT_DATE = 0.5
    WEIGHT_TOPIC = 1

    score_origin = WEIGHT_ORIGIN if candidate_movie['Origin/Ethnicity'] == query_movie['Origin/Ethnicity'] else 0

    score_genre = jaccard_similarity_score(candidate_movie['Genre'], query_movie['Genre']) * WEIGHT_GENRE

    max_diff_years = 5
    date_diff = abs(candidate_movie['Release Year'] - query_movie['Release Year'])
    decay_factor = max(0, 1 - (date_diff / max_diff_years))
    score_date = decay_factor * WEIGHT_DATE

    # query_prob = query_movie['Topic_Probs'].reshape(1, -1)
    # candidate_prob = candidate_movie['Topic_Probs'].reshape(1, -1)
    # topic_similarity = cosine_similarity(query_prob, candidate_prob)[0][0]
    # score_topic = topic_similarity * WEIGHT_TOPIC

    query_embed = query_movie['Embedding'].reshape(1, -1)
    candidate_embed = candidate_movie['Embedding'].reshape(1, -1)
    plot_similarity = cosine_similarity(query_embed, candidate_embed)[0][0]
    score_plot = plot_similarity * WEIGHT_TOPIC

    similarity_score = score_origin + score_genre + score_date + score_topic

    return similarity_score, score_origin, score_genre, score_date, score_topic

def calculate_all_pairwise_scores(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the hierarchical similarity score for every unique pair of movies.
    Returns a long-format DataFrame (Source, Target, Score).
    """
    all_scores = []
    
    # Iterate through every unique pair (i, j) where i < j
    for i in range(len(df)):
        for j in range(i + 1, len(df)):
            movie_i = df.iloc[i]
            movie_j = df.iloc[j]
            
            score, _, _, _, _ = calculate_similarity(movie_i, movie_j)
            
            # Record symmetric scores
            all_scores.append({
                'Source': movie_i['Title'], 
                'Target': movie_j['Title'], 
                'Weight': score
            })
            all_scores.append({
                'Source': movie_j['Title'], 
                'Target': movie_i['Title'], 
                'Weight': score
            })
            
    return pd.DataFrame(all_scores)

In [120]:
all_scores_df = calculate_all_pairwise_scores(df)
all_scores_df.head()

NameError: name 'score_topic' is not defined

In [121]:
def build_recommendation_graph(
    similarity_df: pd.DataFrame, 
    threshold: float = 2.0, 
    top_k: int = 3
):
    """
    Builds a NetworkX graph using the similarity scores as edge weights,
    applying a threshold and a top-K filter for connectivity.
    """
    print(f"Building graph with Threshold={threshold} and Top-K={top_k}...")
    
    # 1. Apply Threshold
    # We only care about scores significantly above 0, which would be dictated by 
    # at least a strong genre match (WEIGHT_GENRE=100) or high date/topic scores.
    # 105.0 requires at least a Genre match (100) + some date/topic score.
    filtered_df = similarity_df[similarity_df['Weight'] >= threshold].copy()

    # 2. Apply Top-K Filter (Top 5 connections for each Source)
    # This keeps the graph sparse and focused on the strongest recommendations.
    filtered_df = filtered_df.sort_values(
        by=['Source', 'Weight'], 
        ascending=[True, False]
    ).groupby('Source').head(top_k)
    
    # 3. Create Graph
    G = nx.Graph()
    
    # Add nodes (all unique movie titles)
    G.add_nodes_from(df['Title'].tolist())
    
    # Add edges with weights
    for _, row in filtered_df.iterrows():
        G.add_edge(row['Source'], row['Target'], weight=row['Weight'])

    print(f"Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    return G

In [122]:
recommendation_graph = build_recommendation_graph(all_scores_df)

from pyvis.network import Network

# Initialize the PyVis network
net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
net.from_nx(recommendation_graph)
net.save_graph("movie_graph.html")

Building graph with Threshold=2.0 and Top-K=3...
Graph created with 673 nodes and 358 edges.


In [ ]:
# top_10_by_title = calculate_hierarchical_similarity(df, 'Kutty')
# print("\n--- TOP 10 SIMILAR MOVIES (Query: Kutty) ---")
# print(df[df['Title'] == 'Cars 2'][['Title', 'Topic', 'Release Year', 'Genre', 'Origin/Ethnicity','Topic_Rep']])
# print(top_10_by_title.to_markdown(index=False))


--- TOP 10 SIMILAR MOVIES (Query: Kutty) ---
        Title  Topic  Release Year                       Genre  \
15994  Cars 2     38          2011  animated, family, spy film   

      Origin/Ethnicity                                          Topic_Rep  
15994         American  [murdered, crime, scene, discovers, met, encou...  
| Title                | Origin/Ethnicity   | Genre           |   Release Year |   Topic |   similarity_score |   score_origin |   score_genre |   score_date |   score_topic |
|:---------------------|:-------------------|:----------------|---------------:|--------:|-------------------:|---------------:|--------------:|-------------:|--------------:|
| Gowravargal          | Tamil              | unknown         |           2010 |      14 |                3.5 |              1 |             1 |          0.5 |  1            |
| Enthiran             | Tamil              | unknown         |           2010 |      11 |                3.5 |              1 |             